In [2]:
import json
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

from google.cloud import storage


In [3]:
BUCKET = "rjr-portal-da-transparencia"
YEAR = 2020

In [5]:
def load_json(name):
    bucket = storage.Client().get_bucket(BUCKET)
    blob = bucket.blob(f'aux_tables/{name}.json')
    maps = json.loads(blob.download_as_string())
    return maps


def mapping(df, map_, column, type_return):
    map_func = udf(lambda key: map_.get(str(key)), type_return())
    df = df.withColumn(column, map_func(col(column)))
    return df

def string_to_date(df, column, year):
    if year > 2014:
        pattern = '%d/%m/%Y'
    else:
        pattern = "%d%b%Y:%H:%M:%S"
    map_func =  udf (lambda date: datetime.strptime(date, pattern) 
                     if type(date) == str 
                     else None, DateType())
    df = df.withColumn(column, map_func(col(column)))
    return df

In [40]:
spark = SparkSession.builder.appName("censo").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

# schema = load_json("matricula_schema")
# try:
#     schema = StructType.fromJson(schema)
# except:
#     schema = StructType.fromJson(json.loads(schema))
df = spark \
    .read \
    .options(header=True, delimiter="|", encoding="utf8") \
    .csv(f"gs://{BUCKET}/landing_zone/censo-escolar/{YEAR}/gestor.csv" )

In [33]:
with open("docentes_schema.json", "w") as file:
    file.write(df.schema.json())

In [16]:
maps = load_json("maps")
string_columns = [column for column in df.columns 
                  if column.startswith("TP") or column.startswith("CO")]

for column in string_columns:
    if column in maps:
        df = mapping(df, maps[column], column, StringType)
    else:
        print(column)

CO_ENTIDADE
CO_DISTRITO


In [41]:
gestor_columns = df.columns

In [76]:
t = [column for column in turmas_columns if column in gestor_columns + docentes_columns + matriculas_columns + escolas_columns] 

In [73]:
m = [column for column in matriculas_columns if column in gestor_columns + docentes_columns + escolas_columns + turmas_columns] 

In [84]:
turmas_em_matricula = [column for column in matriculas_columns if column in turmas_columns]

In [85]:
escolas_em_matricula = [column for column in matriculas_columns if column in escolas_columns]

In [104]:
escolas_ou_turmas_em_matricula =  list(set(escolas_em_matricula + turmas_em_matricula))

In [96]:
escolas_em_turmas =  [column for column in turmas_columns if column in escolas_columns]

In [97]:
escolas_em_turmas

['NU_ANO_CENSO',
 'IN_ESPECIAL_EXCLUSIVA',
 'IN_REGULAR',
 'IN_EJA',
 'IN_PROFISSIONALIZANTE',
 'CO_ENTIDADE',
 'CO_REGIAO',
 'CO_MESORREGIAO',
 'CO_MICRORREGIAO',
 'CO_UF',
 'CO_MUNICIPIO',
 'CO_DISTRITO',
 'TP_DEPENDENCIA',
 'TP_LOCALIZACAO',
 'TP_CATEGORIA_ESCOLA_PRIVADA',
 'IN_CONVENIADA_PP',
 'TP_CONVENIO_PODER_PUBLICO',
 'IN_MANT_ESCOLA_PRIVADA_EMP',
 'IN_MANT_ESCOLA_PRIVADA_ONG',
 'IN_MANT_ESCOLA_PRIVADA_OSCIP',
 'IN_MANT_ESCOLA_PRIV_ONG_OSCIP',
 'IN_MANT_ESCOLA_PRIVADA_SIND',
 'IN_MANT_ESCOLA_PRIVADA_SIST_S',
 'IN_MANT_ESCOLA_PRIVADA_S_FINS',
 'TP_REGULAMENTACAO',
 'TP_LOCALIZACAO_DIFERENCIADA',
 'IN_EDUCACAO_INDIGENA']

In [90]:
escolas_em_gestor =  [column for column in gestor_columns if column in escolas_columns]

In [105]:
a = list(set(escolas_em_turmas).union(escolas_em_gestor))

In [108]:
list(set(escolas_ou_turmas_em_matricula).union(set(a)))

['CO_UF',
 'IN_MANT_ESCOLA_PRIVADA_ONG',
 'TP_TIPO_ATENDIMENTO_TURMA',
 'TP_ETAPA_ENSINO',
 'IN_CONVENIADA_PP',
 'IN_ESPECIAL_EXCLUSIVA',
 'TP_CATEGORIA_ESCOLA_PRIVADA',
 'IN_MANT_ESCOLA_PRIVADA_S_FINS',
 'CO_DISTRITO',
 'IN_EJA',
 'ID_TURMA',
 'TP_LOCALIZACAO_DIFERENCIADA',
 'TP_TIPO_LOCAL_TURMA',
 'CO_REGIAO',
 'CO_MESORREGIAO',
 'IN_MANT_ESCOLA_PRIVADA_EMP',
 'IN_MANT_ESCOLA_PRIVADA_SIND',
 'TP_REGULAMENTACAO',
 'TP_MEDIACAO_DIDATICO_PEDAGO',
 'NU_ANO_CENSO',
 'CO_MUNICIPIO',
 'CO_ENTIDADE',
 'NU_DIAS_ATIVIDADE',
 'IN_MANT_ESCOLA_PRIVADA_OSCIP',
 'IN_MANT_ESCOLA_PRIV_ONG_OSCIP',
 'IN_MANT_ESCOLA_PRIVADA_SIST_S',
 'IN_EDUCACAO_INDIGENA',
 'CO_MICRORREGIAO',
 'TP_DEPENDENCIA',
 'CO_CURSO_EDUC_PROFISSIONAL',
 'IN_REGULAR',
 'TP_CONVENIO_PODER_PUBLICO',
 'TP_LOCALIZACAO',
 'NU_DURACAO_TURMA',
 'IN_PROFISSIONALIZANTE']